For the CF data, box plots showing the output from the TIME pairwise Dynamic Time Warping (DTW) Distance workflow (Workflow 5b) between individual pairs of bacteria grouped by clinical condition were not the best choice, but often one does want to make box plots grouped by a factor variable. With this in mind, this code creates box plots, grouped by condition, of the pairwise DTW distance for the repeated antibiotic perturbation example data available on the TIME website. Dynamic Time Warping is a measure of similarity in longitudinal data, and the TIME version of the algorithm ranges from 0 to 1. 0 is the most similar, and 1 is the most different. For more on the algorithm, see the [relevant research paper here](https://www.frontiersin.org/articles/10.3389/fmicb.2018.00036/full).
<br>

See Create_TDTW_all_example for more details on Workflow 5b and the repeated antibiotic perturbation data set, which shows differences in participant's microbiome before and after antibiotic use. Create_TDTW_all_example generates the input file for this program, merging output files downloaded from the TIME Workflow 5b. However, if you have not run it yet, I have provided the input file for this program, along with the input files for Create_TDTW_all_example, in the 'Extras' folder for your convenience.
<br>

[Click here to run the TIME example analysis](https://web.rniapps.net/time/index.php) now if you would like, or [here to view the antibiotic research paper](https://www.pnas.org/content/108/Supplement_1/4554.long) and learn more about this research. 
<br>

Since the antibiotic example analysis only contains 3 participants, these may not be the most informative of box plots, but they demonstrate the plots well. 

In [ ]:
#import necessary libraries
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#make a list of bacteria which are of special interest to us, which we wish to plot
#these are not the same as the ones of interest in the CF research because they are in the gut rather than the lung
bacteria=['Bacteroides','Roseburia','Clostridium']

## Section 1

The first few cells modify the dataset to focus on only 3 OTU's (operational taxonomic units) in our list, creating a separate file for each of the three which shows the distance from the other OTU's to that one OTU for every condition. Three similar files were created for just the DTW distances which were caluclated for the 'All' condition in the programs DTW_All_boxplots and DTW_All_boxplots_example. Of course, you can focus on any number of OTU's by editing the list.
<br>

If the filtered files have already been created and saved for the bacteria you wish to plot, you can skip to Section 2 and read them in.
<br>

The first cell in this section reads in the file created in the program Create_TDTW_all_example. If you have not run that program yet, edit the cell as directed in the comments to use the version in the 'Extras' folder. 

In [ ]:
#read in the file, created in the program Create_TDTW_all_example, which contains all the output from Workflow 5b
df = pd.read_csv('Data/TDTW_all_example.csv')
#to use the ones in the 'Extras' folder, comment out the previous line and un-comment this one:
#df = pd.read_csv("Data/Extras/TDTW_all_example.csv")

In [ ]:
#rename the first column, which has no name, to 'Sample' 
df.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)

In [ ]:
#extract the participant's clinical status from the sample identifier and change to title case
#this particular example output is actually already in title case 
#I included str.title() for the convenience of those editing it to use on data with a lower case status (like the CF data)
df['Status']=df['Sample'].str.split('_').str[2].str.title()

In [ ]:
#extract the participant's ID number from the identifier
df['ID']=df['Sample'].str.split('_').str[1]
#review the changes to confirm that they worked
df.head()

In [ ]:
#keep only the colums with at least 5 non-NaN's
df.dropna(thresh=5, axis=1, inplace=True)
#you can, of course, change the threshold number based on your data, skip this cell and leave all NaN's, 
#or drop all NaN's with df3.dropna(axis=1, inplace=True) 
#view which columns were omitted, if desired
#df.head()

In Create_TDTW_all_example, we chose to add a single underscore as a separator between bacteria names when we made the file TDTW_all_example. This means that, if you choose to edit this code to run it on your own data instead of the example data, the code below may need to be modified to reflect formatting differences in your data.

In [ ]:
#make a list of columns (pairs of OTU's) for only the 3 OTU's of interest, excluding the ones that pair them with themselves
#we also include the 'Status' column to enable grouping by status
clist=[]
for i in range(len(bacteria)):
    #in this line, you may need to do a minor edit such as removing some underscores if your bacteria names don't have them
    #viewing the head of the table above should tell you if and how you need to change the str.format() function
    j=[col for col in df.columns if col.startswith('{}'.format(bacteria[i]))
       and col!= '{}_{}'.format(bacteria[i],bacteria[i]) or col=='Status']
    clist.append(j)

Run the next cell as it is written if you want to save the files after creating them. That will enable you to skip Section 1 and go straight to Section 2 the next time you run this program. If you don't want to save the files, just comment out the line which saves them.

In [ ]:
#create the files and save, adding them to a dictionary
selectdfs={}
for i in range(len(bacteria)):
    df2=df.filter(clist[i], axis=1)
    #since each data frame is for the distance from one bacteria, change the column names to be just the second bacteria
    #this code may need adjustment depending on how many underscores are in your bacteria names
    df2.columns=[w.replace('{}_'.format(bacteria[i]),'') for w in df2.columns]
    #save file if you want to skip Section 1 next time, or comment out the next line
    df2.to_csv("Data/TDTW_{}_conditions_boxplots.csv".format(bacteria[i]), index=False)
    selectdfs[bacteria[i]]=df2

## Section 2

If you ran Section 1, you can skip the first cell in Section 2, which reads the files in if you have saved them previously and creates the dictionary which was made in Section 1's last cell. 

In [ ]:
#if you saved the files, read them into a dictionary here
#if you ran Section 1, you already have the dictionary, so skip this cell
selectdfs = {i: pd.read_csv("Data/TDTW_{}_conditions_boxplots.csv".format(i)) for i in bacteria}
#view one of the files to get a feel for what the data look like, if desired
#selectdfs['Bacteroides'].head()

All of my plotting functions are written to save the plots to a folder called 'Plots,' which is in this repository as well. Adjust the file path if you want to save them somewhere else, or comment out the line of code which saves them if you prefer not to save.

In [ ]:
#now we plot them, taking care not to plot the same pairwise distance twice
skip=[]
for i in bacteria:
    clist=list(selectdfs[i].columns.values)
    clist=[col for col in clist if col!='Status' and col not in skip]
    for j in clist:
        plt.figure(figsize=(10,8))
        sns.boxplot(x='Status',y='{}'.format(j),data=selectdfs[i], order=['Precp','Firstcp','Firstwpc','All'])
        plt.title('DTW Distance between {} and {}'.format(i,j), size=16)
        plt.xlabel('Status', size=14)
        plt.ylabel('Distance',size=14)
        #save file if desired
        plt.savefig("Plots/{}_{}.png".format(i,j), format = "png")
        plt.show()
    skip.append(i) 

Sample of what one box plot will look like:
<br>
<img src='https://imgur.com/OTctuR0.png' style:height='150px'>
